In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time 
from openpyxl import load_workbook
import os.path 
import psycopg2
import pgdb

hostname = '10.195.25.10'
username = 'postgres'
password = '123'
database = 'musicbrainz'
port='54587'
cid ="a154f4842df643a99f9057fb741c86e0" 
secret = "543080531fce4f30be3da2c36782ace1"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

today_date=(time.strftime("%Y-%m-%dT%H:%M:%SZ"))
date=(time.strftime("%d/%m/%Y"))

def doQuery( conn ) :
    
    cur = conn.cursor()
    cur.execute( """SELECT a.id as id_msbz, a.name,art.name as artist_type , g.name as gender,
    arcity.name as artist_city,arcoutry.name as  artist_country  
    FROM musicbrainz.artist a , musicbrainz.area arcity, musicbrainz.area_type at ,
    musicbrainz.artist_type art , musicbrainz.area arcoutry ,musicbrainz.gender g
    where  a.type=art.id and a.begin_area=arcity.id and a.area=arcoutry.id  and a.gender=g.id 
    """)
    return cur.fetchall()
 
    
artist_musicbrainz =[]
myConnection = psycopg2.connect( host=hostname, user=username, password=password, dbname=database,port=port )
artist_musicbrainz =doQuery( myConnection )
myConnection.close()
# print size list



#Verify if File in certain date already exists
filename='D:\dproject_bdma2018\ETL\data\Artistsout'+today_date+'.xlsx'
if os.path.exists(filename):
    print(filename+' '+'exists')
else: 
    # create empty lists where the results are going to be stored
    artist_name = []
    artist_genre= []
    artist_popularity = []
    artist_id = []
    artist_followers = []
    artist_id_mbz = []
    artist_type =[]
    artist_gender =[]
    artist_city =[]
    artist_country =[]
 
    for i in range(0,len(artist_musicbrainz)-1,1):
        for id_msbz,name,artist_type,gender,artist_city,artist_country in artist_musicbrainz :
            track_results = sp.search(q=name, type='artist',market='FR', limit=50)
           # print (track_results)
            
            if  not track_results[t]['items'] :
                print(track_results[t]['items'])
                print('artiste non trouvé')
        
            else : 
                #print (track_results)
                for i, t in enumerate(track_results):
                    print (track_results[t]['items'][0]['name'])                    
                    artist_name.append(track_results[t]['items'][0]['name'])
                    artist_genre.append(track_results[t]['items'][0]['genres'])
                    artist_popularity.append(track_results[t]['items'][0]['popularity'])
                    artist_id.append(track_results[t]['items'][0]['id'])
                    artist_followers.append(track_results[t]['items'][0]['followers']['total'])
                    artist_x.append(x)
                    artist_id_mbz.append(id_msbz)
                    artist_type.append(artist_type)
                    artist_gender.append(gender)
                    artist_city.append(artist_city)
                    artist_country.append (artist_country)
                    
                    
                    print ('finish')
                print ('finish for')
                df_tracks = pd.DataFrame({'artist_name':artist_name,
                                          'artist_genre':artist_genre,
                                          'artist_popularity':artist_popularity,
                                          'artist_id.':artist_id,
                                          'Date':date,
                                          'artist_followers':artist_followers,
                                          'artist_id_mbz':artist_id_mbz,
                                          'artist_type':artist_type,
                                          'artist_gender':artist_gender,
                                          'artist_city':artist_city,
                                          'artist_country':artist_country
                                         })

  
                  #Create xlsx File 
                writer = pd.ExcelWriter(filename, engine='xlsxwriter')
                df_tracks.to_excel(writer,encoding='utf8')
                writer.save()
        


24942
